In [1]:
!pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11757 sha256=57c3cd8278c0a739a1f5a9d09e413e58e50804a066a75663858a33137c6e7a81
  Stored in directory: /Users/ajay/Library/Caches/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Wikipedia Retrievers

In [1]:
from langchain_community.retrievers import WikipediaRetriever

In [2]:
retriever = WikipediaRetriever(
    top_k_results=2, lang="en"
)

In [7]:
query = "What is the history behind the Spider-Man as a superhero?"

In [8]:
docs = retriever.invoke(query)

In [12]:
docs[0].metadata

{'title': 'Spider-Man in film',
 'summary': "Spider-Man has appeared in film since the 1977 made-for-television movie broadcast on CBS. The motion picture rights to Spider-Man belonged to Marvel Entertainment until 1999, when Sony Pictures bought them for $7 million and produced Spider-Man's first theatrical film. He has been Marvel's most successful character in the American film industry ever since.\nAfter selling the Spider-Man motion picture rights to Sony, Marvel eventually founded its own film studio, Marvel Studios, developing the Marvel Cinematic Universe (MCU) shared universe based only on the characters they still held the rights to. This would change in 2016, when Sony and the Walt Disney Company entered an agreement to introduce Spider-Man to the MCU. Despite some disagreements pertaining to finances and merchandising between the two parties, the agreement proved to be a successful endeavor for both companies. The crossover films Avengers: Infinity War and Avengers: Endgame

In [11]:
for i,doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content: \n {doc.page_content} ...")
    print(f"MetaData: \n {doc.metadata} ...")


--- Result 1 ---
Content: 
 Spider-Man has appeared in film since the 1977 made-for-television movie broadcast on CBS. The motion picture rights to Spider-Man belonged to Marvel Entertainment until 1999, when Sony Pictures bought them for $7 million and produced Spider-Man's first theatrical film. He has been Marvel's most successful character in the American film industry ever since.
After selling the Spider-Man motion picture rights to Sony, Marvel eventually founded its own film studio, Marvel Studios, developing the Marvel Cinematic Universe (MCU) shared universe based only on the characters they still held the rights to. This would change in 2016, when Sony and the Walt Disney Company entered an agreement to introduce Spider-Man to the MCU. Despite some disagreements pertaining to finances and merchandising between the two parties, the agreement proved to be a successful endeavor for both companies. The crossover films Avengers: Infinity War and Avengers: Endgame included Spider-

### VectorStore Retriever


In [13]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [14]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [15]:
# Step 2: Initialize your embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# Step 3: Create chroma vector store in memory
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="retriver_collection"
    
)

/Users/ajay/Documents/Personal/Courses/LangChain-CampusX/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
# Step 4: Convert vectorstore into a retriever
retriever=vector_store.as_retriever(search_kwargs={"k":2})

In [17]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [18]:
results

[Document(metadata={}, page_content='Chroma is a vector database optimized for LLM-based search.'),
 Document(metadata={}, page_content='LangChain helps developers build LLM applications easily.')]

In [20]:
for i,doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(f"Content: {doc.page_content}")


--- Result 1 ---
Content: Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Content: LangChain helps developers build LLM applications easily.


### MMR


In [21]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [22]:
from langchain.vectorstores import FAISS

# Step 1: Initialize embedding model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# Step 2: Create the FAISS vector store from documents
vector_store = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [ ]:
# Enable MMR Retriever
retriever = vector_store.as_retriever(
    search_type="mmr", #This Enables MMR 
    search_kwargs={"k": 3, "lambda_mult": 0.5} # k = top results, lambda_mult = relevance-diversity balance (range 0-1)  
)

In [24]:
query = "what is Langchain?"
results = retriever.invoke(query)

In [27]:
for i, doc in enumerate(results):
    print(f"\n Result: {i+1} ")
    print(f"\n Content: {doc.page_content}")


 Result: 1 

 Content: LangChain supports Chroma, FAISS, Pinecone, and more.

 Result: 2 

 Content: LangChain is used to build LLM based applications.

 Result: 3 

 Content: Embeddings are vector representations of text.


### Multi-Query Retriever

In [54]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
load_dotenv()

True

In [55]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [56]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# Create FAISS vector store
vector_store = FAISS.from_documents(
    embedding=embedding_model,
    documents=all_docs
)

In [57]:
# Create Retriever
similarity_retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [58]:
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
    llm=ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-8b",
        api_key=os.environ["GEMINI_API_KEY"]
    )
)

In [59]:
query = "How to improve energy levels and maintain balance?"

In [60]:
similarity_result = similarity_retriever.invoke(query)
multiquery_result = multi_query_retriever.invoke(query)

In [62]:
multiquery_result

[Document(id='162ae0f7-28b6-45dc-8881-7a8a64eef8ba', metadata={'source': 'H5'}, page_content='Drinking sufficient water throughout the day helps maintain metabolism and energy.'),
 Document(id='5f5c9391-b6bd-494a-9e20-2c089b0c4dbb', metadata={'source': 'H2'}, page_content='Consuming leafy greens and fruits helps detox the body and improve longevity.'),
 Document(id='32b46404-3122-4084-816e-fcf1df817038', metadata={'source': 'H4'}, page_content='Mindfulness and controlled breathing lower cortisol and improve mental clarity.'),
 Document(id='1be3bb90-1d51-41b1-b7e1-59c3df1bcd4e', metadata={'source': 'I1'}, page_content='The solar energy system in modern homes helps balance electricity demand.'),
 Document(id='9c56fa5a-3192-4b56-ba54-a944bedcd91e', metadata={'source': 'I3'}, page_content='Photosynthesis enables plants to produce energy by converting sunlight.'),
 Document(id='76980d81-444d-4f22-8857-6ecf94e4bbdc', metadata={'source': 'H1'}, page_content='Regular walking boosts heart healt

In [61]:
for i, doc in enumerate(similarity_result):
    print(f"\n --- Result {i+1} ---")
    print(doc.page_content)
    
print("*"*100)

for i, doc in enumerate(multiquery_result):
    print(f"\n --- Result {i+1} ---")
    print(doc.page_content)


 --- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 --- Result 2 ---
The solar energy system in modern homes helps balance electricity demand.

 --- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

 --- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 --- Result 5 ---
Photosynthesis enables plants to produce energy by converting sunlight.
****************************************************************************************************

 --- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

 --- Result 2 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

 --- Result 3 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

 --- Result 4 ---
The solar energy system in modern homes helps balance electricity demand.

 --- Result 5 ---
Photosy

In [67]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document
from dotenv import load_dotenv
import os
load_dotenv()

True

In [68]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [69]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# Create FAISS vector store
vector_store = FAISS.from_documents(
    embedding=embedding_model,
    documents=docs
)

In [70]:
base_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [72]:
# Set up the compressor using an LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-8b",
    api_key=os.environ["GEMINI_API_KEY"]
)
compressor = LLMChainExtractor.from_llm(llm=llm)

In [73]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [74]:
compressed_result = compression_retriever.invoke("What is Photosynthesis?")

In [81]:
print(type(compressed_result))

<class 'list'>


In [89]:
for i, doc in enumerate(compressed_result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.

--- Result 3 ---
Photosynthesis does not occur in animal cells.
